<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/master/landlab_header.png"></a>

# The deAlmeida Overland Flow Component 

<hr>
<small> For instructions on how to run an interactive iPython notebook, click here: <a href="https://github.com/landlab/tutorials/blob/master/README.md">https://github.com/landlab/tutorials/blob/master/README.md</a></small><br>
<small>For the unexpanded version to download and run, click here: <a href="http://nbviewer.ipython.org/github/landlab/tutorials/blob/master/overland_flow/overland_flow_driver_unexpanded.ipynb">http://nbviewer.ipython.org/github/landlab/tutorials/blob/master/overland_flow/overland_flow_driver_unexpanded.ipynb</a></small><br>
<small>For more Landlab tutorials, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

This notebook illustrates running the deAlmeida overland flow component in an extremely simple-minded way on a real topography, then shows it creating a flood sequence along an inclined surface with an oscillating water surface at one end.

First, import what we'll need:

In [1]:
from __future__ import print_function

from landlab.components.overland_flow import OverlandFlow
from landlab.grid.structured_quad import links
from landlab.plot.imshow import imshow_node_grid
from landlab import RasterModelGrid
from landlab.io.esri_ascii import read_esri_ascii
from matplotlib.pyplot import figure, show
import numpy as np
from time import time
%matplotlib inline

Pick the initial and run conditions

In [2]:
run_time = 100           # duration of run, (s)
h_init = 0.1            # initial thin layer of water (m)
n = 0.01                  # roughness coefficient, (s/m^(1/3))
g = 9.8                   # gravity (m/s^2)
alpha = 0.7               # time-step factor (nondimensional; from Bates et al., 2010)
u = 0.4                   # constant velocity (m/s, de Almeida et al., 2012)
run_time_slices = (10,50,100,200)

Elapsed time starts at 1 second. This prevents errors when setting our boundary conditions.

In [3]:
elapsed_time = 1.0

Use Landlab methods to import an ARC ascii grid, and load the data into the field that the component needs to look at to get the data. This loads the elevation data, z, into a "field" in the grid itself, defined on the nodes.

In [4]:
rmg, z = read_esri_ascii('HalfFork.asc')
rmg.add_field('node', 'topographic__elevation', z)
rmg.set_nodata_nodes_to_closed(z, -9999.)

We can get at this data with this syntax:

In [5]:
np.all(rmg.at_node['topographic__elevation'] == z)

True

Note that the boundary conditions for this grid mainly got handled with the final line of those three, but for the sake of completeness, we should probably manually "open" the outlet. We can find and set the outlet like this:

In [6]:
core_node_order = np.argmin(z[rmg.core_nodes])
my_outlet_node = rmg.core_nodes[core_node_order]
rmg.status_at_node[my_outlet_node] = 1  # 1 is the code for fixed value

Now initialize a couple more grid fields that the component is going to need:

In [7]:
rmg.create_node_array_zeros('water_depth')                   # water depth (m)
rmg.add_zeros('active_link', 'water_discharge')              # unit discharge (m2/s)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [8]:
rmg.at_node['water_depth'] += h_init

Now instantiate the component itself

In [9]:
of = OverlandFlow(rmg)

/Users/jennyknuth/anaconda/lib/python2.7/site-packages/landlab/components/overland_flow/generate_overland_flow_deAlmeida.py:107: UserWarning: No input file provided! Default file and default values will be used
  "No input file provided! Default file and default values "


IOError: [Errno 2] No such file or directory: '/Users/jennyknuth/anaconda/lib/python2.7/site-packages/landlab/components/overland_flow/overland_flow_input.txt'

Now we're going to run the loop that drives the component:

In [ ]:
while elapsed_time < run_time:
    # First, we calculate our time step.
    dt = of.gear_time_step(rmg)
    print(dt)
    # Now, we can generate overland flow.
    of.overland_flow(rmg, dt)
    # Increased elapsed time
    elapsed_time += dt

In [ ]:
imshow_node_grid(rmg, 'water_depth')

Now let's get clever, and run a set of time slices:

In [ ]:
elapsed_time = 1.
for t in run_time_slices:
    while elapsed_time < t:
        # First, we calculate our time step.
        dt = of.gear_time_step(rmg)
        # Now, we can generate overland flow.
        of.overland_flow(rmg, dt)
        # Increased elapsed time
        elapsed_time += dt
    figure(t)
    imshow_node_grid(rmg, 'water_depth', vmax=14.)

### Click here for more <a href="https://github.com/landlab/landlab/wiki/Tutorials">Landlab tutorials</a>